In [ ]:
import numpy as np

In [ ]:
path = "instances/instance_cars.txt"
f = open (path, "r")

In [ ]:
f

In [ ]:
line = f.readline()
count = int(line)
print (count)

In [ ]:
# 本函数用作将instances文件夹中的实例文件读取，并将之转化为list和nparray变量

def dataLoading (dname):
	# 根据实例名称读入相应实例
	path = "instances/instance_"
	f = open (path + dname + ".txt", "r")

	# 读入实例中样例的总个数 
	line = f.readline()
	count = int(line)

	# 初始化返回变量
	instances = list()
	instance = list()

	# 通过循环第一次读取一行，并将样例转化为nparray，存入list中
	for i in range (count):
	    instance.clear()
	    n, m = [int (item) for item in f.readline().split()]
	    for j in range (n):
	        line = [int (item) for item in f.readline().split()]
	        line = line[1::2]
	        instance.append (line.copy())
	    instances.append (np.array (instance.copy(), dtype = 'int32'))
	   
	# 初始化标准结果上界与下界
	upper_bounds = list()
	lower_bounds = list()

	# 读取上界、下界
	f.readline()
	for i in range (count):
	    upper_bounds.append (int (f.readline()))
	upper_bounds = np.array (upper_bounds, dtype = 'int32')
	f.readline()
	for i in range (count):
	    lower_bounds.append (int (f.readline()))
	lower_bounds = np.array (lower_bounds, dtype = 'int32')

	return instances, upper_bounds, lower_bounds

In [ ]:
# dataLoading ('vrf_large')

In [1]:
import numpy as np
from utils.utils import utils

In [4]:
class Solver (object):
    def __init__ (self, data):
        self.n = len (data)
        self.m = len (data[0])
        self.makespan = 0
        self.idle_time = 0
        
        self.permutation = list()
        self.data = data
        self.idle_times = np.zeros ((self.n), dtype = 'int32')
        
    def calculateCompletionTimes (self):
        permutation = np.array (self.permutation, dtype = 'int32')
        completion_times = utils.calculateCompletionTimes (permutation, self.data, self.m, 1)
        return np.array (completion_times)
    
    def calculateMakespan (self):
        permutation = np.array (self.permutation, dtype = 'int32')
        self.makespan = utils.calculateCompletionTimes (permutation, self.data, self.m, 0)
        return self.makespan
    
    def insertIntoBestPosition (self, inserted_job, tie_breaking = False):
        use_tie_breaking = 1 if tie_breaking == True else 0
        permutation = np.array (self.permutation, dtype = 'int32')
        best_position, self.makespan = utils.acceleration (permutation, self.data, inserted_job, self.m, use_tie_breaking)
        self.permutation.insert (best_position - 1, inserted_job)
        return self.makespan
    
    def calculateIdleTimes (self):
        permutation = np.array (self.permutation, dtype = 'int32')
        idle_times = utils.calculateIdleTimes (permutation, self.data, self.m)
        self.idle_times = np.array (idle_times)

In [5]:
class Population (object):
    def __init__ (self, data, p = 50):
        self.n = len (data)
        self.m = len (data[0])
        self.p = p
        self.data = data
        self.chromes = list()
        self.makespans = list()
        self.idle_times = list()
        self.best_makespan = 0
        self.best_chrome = list()
        self.best_index = -1
        
    def calculateMakespans (self):
        chromes = np.array (self.chromes, dtype = 'int32')
        self.makespans = utils.calculateMakespans (chromes, self.data, self.n, self.m, self.p)
        return np.array (self.makespans)
    
    def calculateBestCombination (self):
        self.best_index = np.argmin (self.makespans) 
        return self.best_index
    
    def calculateBestChrome (self):
        self.best_index = self.calculateBestCombination()
        self.best_chrome = self.chromes[self.best_index]
        return np.array (self.best_chrome)
    
    def calculateBestMakespan (self):
        self.best_index = self.calculateBestCombination()
        self.best_makespan = self.makespans[self.best_index]
        return self.best_makespan


In [6]:
def initializeChromes (n, p, initialize_method = 'random'):
    if initialize_method == 'random':
        permutation = np.arange (1, n + 1)
        chromes = list()
        for i in range (p):
            random.shuffle (permutation)
            chromes.append (permutation.copy())
    return chromes

def selectChromes (chromes, fitness, tournament_size = 5, select_method = 'tournament'):
    p = len (chromes)
    if select_method == 'tournament':
        population = [x + 1 for x in range (len(fitness))]
        permutation = []
        for dummy in range (p):
            aspirants = random.sample (population, tournament_size)
            best = -1
            for asp in aspirants:
                if fitness[asp - 1] > best:
                    best = fitness[asp - 1]
                    winner = asp
            if best == 0:
                winner = random.choice (aspirants)
            permutation.append (winner)
    elif select_method == 'roulette':
        total_fit = np.sum (fitness)
        permutation = []
        for i in range (p):
            rand = random.random() * total_fit
            for k in range (len (fitness)):
                rand -= fitness[k]
                if rand <= 0:
                    permutation.append (k + 1)
                    break
            else:
                permutation.append (len (fitness) - 1)
    elif select_method == 'stochastic_uni':
        distance = np.sum (fitness) / p
        start = random.uniform(0, distance)
        points = [start + i * distance for i in range (p)]
        permutation = []
        for point in points:
            i = 0
            total_fitness = fitness[i]
            while total_fitness < point:
                i += 1
                total_fitness += fitness[i]
            permutation.append (i + 1)
    return [chromes[x - 1] for x in permutation]

def crossChromes (chromes, cross_rate):
    p = len (chromes)
    n = len (chromes[0])
    for i in range (0, p, 2):
        if random.random() < cross_rate:
            j = (i + 1) % n
            dic1 = dict (zip (chromes[i], range (n)))
            dic2 = dict (zip (chromes[j], range (n)))
            spl = np.array(random.sample(range(0, n + 1), 2), dtype = 'int32')
            r = np.min (spl)
            s = np.max (spl)
            diff1 = list (set (chromes[i][r:s]) - set (chromes[j][r:s]))
            diff2 = list (set (chromes[j][r:s]) - set (chromes[i][r:s]))
            for index, x in enumerate(diff2):
                chromes[i][dic1[x]] = diff1[index]
                chromes[j][dic2[diff1[index]]] = x
            tmp = chromes[i][r:s].copy()
            chromes[i][r:s] = chromes[j][r:s].copy()
            chromes[j][r:s] = tmp.copy()
    # return chromes

        
def muteChromes (chromes, mute_rate, mute_method):
    p = len (chromes)
    for i in range (p):
        if random.random() < mute_rate:  
            if mute_method == 'interchange':
                muteInterchange (chromes[i])
            elif mute_method == 'reverse':
                muteReverse (chromes[i])
            elif mute_method == 'insertion':
                muteInsertion (chromes[i])
            elif mute_method == 'mixed':
                seq = random.randint (1, 7)
                if seq % 2 == 1:
                    muteInsertion (chromes[i])
                elif (seq / 2) % 2 == 1:
                    muteReverse (chromes[i])
                elif (seq / 4) % 2 == 1:
                     muteInterchange (chromes[i])
            # elif mute_method == 'heuristic':
               
def muteInterchange (chrome):
    r, s = np.array(random.sample(range(0, len(chrome)), 2), dtype = 'int32')
    tmp = chrome[r]
    chrome[r] = chrome[s]
    chrome[s] = tmp
    
def muteReverse (chrome):
    slp = np.array(random.sample(range(0, len(chrome) + 1), 2), dtype = 'int32')
    r = np.min (slp)
    s = np.max (slp)
    p = - len(chrome) - 1 + s if r == 0 else s - 1
    q = - len(chrome) - 1 if r == 0 else r - 1 
    chrome[r:s] = chrome[p:q:-1]
    
def muteInsertion (chrome):
    slp = np.array(random.sample(range(0, len(chrome)), 2), dtype = 'int32')
    r = np.min (slp)
    s = np.max (slp)
    tmp = chrome[r]
    chrome[r:(s - 1)] = chrome[(r + 1):s]
    chrome[s - 1] = tmp

In [7]:
from localSearch import localSearchPop

In [15]:
class GeneticAlgorithm (object):
    def __init__ (self, data, tie_breaking = False, local_optimum = True, local_search = False, 
                  select_method = 'tournament', tournament_size = 5, initialize_method = 'random', 
                  mute_method = 'mixed', cross_rate = 0.6, mute_rate = 0.8, p = 50, max_generation = 500,
                  relax_rate = 1.2):
        self.p = p
        self.n = len (data)
        self.m = len (data[0])
        self.max_generation = max_generation
        self.current_pop = Population (data, self.p)
        self.new_pop = Population (data, self.p)
        self.best_pop = Population (data, self.p)
        self.tie_breaking = tie_breaking
        self.local_optimum = local_optimum
        self.local_search = local_search
        self.select_method = select_method
        self.initialize_method = initialize_method
        self.mute_method = mute_method
        self.cross_rate = cross_rate
        self.mute_rate = mute_rate
        self.tournament_size = tournament_size
        self.relax_rate = relax_rate
        
    def eval (self, runtime):
        self.iterations = 0
        time_limit = datetime.now() + timedelta (milliseconds = runtime)
        self.current_pop.chromes = initializeChromes (self.n, self.p, initialize_method = self.initialize_method)
        if self.local_search == True:
            localSearchPop (self.current_pop, self.local_optimum, self.tie_breaking)
        self.current_pop.calculateMakespans()
        self.best_pop.best_chrome = self.current_pop.calculateBestChrome()
        self.best_pop.best_makespan = self.current_pop.calculateBestMakespan()
        print ("BEST", self.best_pop.best_makespan)
        self.best_pop.chromes = self.current_pop.chromes.copy()
        # print (self.best_pop.chromes, self.best_pop.best_makespan)
        for generation in range (self.max_generation):
            if datetime.now() >= time_limit:
                break
            self.new_pop.chromes = self.current_pop.chromes.copy()
            self.new_pop.calculateMakespans()
            makespan = np.max (np.array (self.new_pop.makespans)) * self.relax_rate - np.array (self.new_pop.makespans)
            selectChromes (self.new_pop.chromes, makespan, tournament_size = self.tournament_size, select_method = self.select_method)
            crossChromes (self.new_pop.chromes, self.cross_rate)
            muteChromes (self.new_pop.chromes, self.mute_rate, self.mute_method)
            if self.local_search == True:
                localSearchPop (self.new_pop, self.local_optimum, self.tie_breaking)
            self.new_pop.calculateMakespans()
            self.new_pop.best_makespan = self.new_pop.calculateBestMakespan()
            self.new_pop.best_chrome = self.new_pop.calculateBestChrome()
            if self.new_pop.best_makespan < self.current_pop.best_makespan:
                self.current_pop.chromes = self.new_pop.chromes
                self.current_pop.best_makespan = self.new_pop.best_makespan
                self.current_pop.best_chrome = self.new_pop.best_chrome
                if self.current_pop.best_makespan < self.best_pop.best_makespan:
                    self.best_pop.chromes = self.current_pop.chromes
                    self.best_pop.best_makespan = self.current_pop.best_makespan
                    self.best_pop.best_chrome = self.current_pop.best_chrome
            self.iterations += 1
        

In [29]:
def ex6():
    instances, _, _ = loadData.dataLoading ("bit")
    for instance in instances:
        ig = GeneticAlgorithm(instance, max_generation = 50000, local_search = False, p = 30, cross_rate = 0.8, mute_rate = 0.15, mute_method = 'reverse')
        ig.eval(10000)
        print("Best makespan", ig.best_pop.best_makespan, ig.iterations)
        print("Job permutation:", ig.best_pop.best_chrome)

In [30]:
"""
    朴素局部搜索算法的实现
"""
import random
from solver import Solver
import numpy as np

def localSearch(solver, local_optimum = True, tie_breaking = False):
    current_makespan = 0
    need_improve = True
    MAXLOOP = 1000000
    for dummy in range (MAXLOOP):
        if need_improve == False:
            break
        current_jobs = solver.permutation.copy()
        random.shuffle (current_jobs)
        for job in current_jobs:
            solver.permutation.remove (job)
            current_makespan = current_makespan if current_makespan != 0 else solver.calculateMakespan()
            solver.insertIntoBestPosition (job, tie_breaking)
            if solver.makespan < current_makespan:
                need_improve = True
                current_makespan = solver.makespan
            else:
                need_improve = False
        if local_optimum == False:
            break
    return
            
def localSearchPop (population, local_optimum = True, tie_breaking = False):
    p = population.p
    for index in range (p):
        solver = Solver (population.data)
        solver.permutation = list(population.chromes[index].copy())
        localSearch(solver, local_optimum, tie_breaking)
        population.chromes[index] = solver.permutation.copy()

In [31]:
ex6()

BEST 7685
Best makespan 7038 12713
Job permutation: [ 8  3  5 11  1  9  2  7  4 10  6]
BEST 8703
Best makespan 8366 12806
Job permutation: [7 3 8 5 2 1 6 4]
BEST 8123
Best makespan 7166 11673
Job permutation: [ 7  3  4 11 13  2  8 12 10  9  6  1  5]
BEST 8230
Best makespan 7399 10735
Job permutation: [11 12  5  6 10  4  3  2  9  8  7  1]
BEST 8772
Best makespan 8003 11419
Job permutation: [ 4 12 13  6  9 10  7  1 14  8  3 11  5  2]
BEST 8778
Best makespan 7750 11981
Job permutation: [ 6  2  3  1  4  8 10  9  7  5]
BEST 1688
Best makespan 1556 10659
Job permutation: [14 19  4 16 13  8 11 20  9 12  7  3  2 15  6  1 18  5 10 17]
BEST 2154
Best makespan 2082 10233
Job permutation: [11 20  5 15  9  1 17  6 14  7 16  8 12  3 19 10 13 18  2  4]
BEST 1229
Best makespan 1169 10545
Job permutation: [ 1  8 11  7  2  3 13 14  9  6  4 17 12 16 10  5 19 15 18 20]
BEST 2225
Best makespan 2087 9984
Job permutation: [18 12  5  8  2  6 19 17 13 16 15 11 20 10  3  7  9  4  1 14]
BEST 3792
Best makespan 3

In [ ]:
chrome = [1, 2, 3, 4, 5, 6, 7, 8]
muteInterchange (chrome)
muteReverse (chrome)
print(chrome)
muteInsertion (chrome)
chrome

In [ ]:
random.randint(1, 4)

In [2]:
from loadData import dataLoading
datas, _, _ = dataLoading ('cars')

In [ ]:
pop = Population (datas[0])

In [ ]:
pop.chromes = initializeChromes (pop.n, pop.p)

In [ ]:
pop.calculateMakespans(), pop.chromes

In [ ]:
pop.calculateBestCombination(), pop.calculateBestChrome(), pop.calculateBestMakespan()

In [ ]:
sol = Solver(datas[0])
sol.permutation = pop.calculateBestChrome()
sol.calculateMakespan()

In [ ]:
makespan = np.max (np.array (pop.makespans)) * 1.2 - np.array (pop.makespans)
makespan, np.sum (np.array (pop.makespans))

In [ ]:
chromes0 = selectChomes (pop.chromes, makespan, select_method = 'tournament')
chromes1 = selectChomes (pop.chromes, makespan, select_method = 'roulette')
chromes2 = selectChomes (pop.chromes, makespan, select_method = 'stochastic_uni')

In [ ]:
pop.chromes = chromes0
pop.calculateMakespans()
print (np.sum (np.array (pop.makespans)))
pop.chromes = chromes1
pop.calculateMakespans()
print (np.sum (np.array (pop.makespans)))
pop.chromes = chromes2
pop.calculateMakespans()
print (np.sum (np.array (pop.makespans)))

In [ ]:
crossChromes (pop.chromes, 0.5)

In [ ]:
pop.chromes

In [ ]:
muteChromes (pop.chromes, 0.5, mute_method = 'mixed')

In [ ]:
solver = Solver(datas[0])

In [ ]:
solver.permutation.append (1)
solver.permutation.append (2)

In [ ]:
solver.insertIntoBestPosition(4)

In [ ]:
solver.makespan

In [ ]:
solver.permutation

In [ ]:
import random

In [ ]:
"""
    NEH算法的实现, 注意这里使用了三种不同的初始化方法: 
    SD : 按照每道工序单独在不同机器上消耗时间总和的倒序初始化
    AD : 按照

"""
def NEH (solver, tie_breaking = False, order = 'SD'):
    if order == 'SD':
        jobs = sd_order (solver)
    elif order == 'AD':
        jobs = ad_order (solver)
    else:
        jobs = [x for x in range (1, solver.n + 1)]
        random.shuffle (jobs)
    solver.permutation = jobs[:2]
    ms1 = solver.calculateMakespan()
    solver.permutation = jobs[1::-1]
    if ms1 < solver.calculateMakespan():
        solver.permutation = jobs[:2]
        solver.makespan = ms1
    for job in jobs[2:]:
        solver.insertIntoBestPosition (job, tie_breaking)
        
def sd_order (solver):
    total_processing_times = dict()
    for i in range(1, solver.n + 1):
        total_processing_times[i] = np.sum(solver.data[i - 1])

    return sorted(total_processing_times, key = total_processing_times.get, reverse = True)


def ad_order (solver):
    average_plus_deviation = dict()
    for i in range(1, solver.n + 1):
        avg = np.mean(solver.data[i - 1])
        dev = np.std(solver.data[i - 1])
        average_plus_deviation[i] = avg + dev
    return sorted(average_plus_deviation, key = average_plus_deviation.get, reverse = True)

In [ ]:
datas, up, low = dataLoading ('vrf_large')

In [ ]:
ers = list()
for i, data in enumerate (datas):
    solver = Solver (data)
    NEH (solver)
    ms = solver.calculateMakespan()
    ers.append (np.absolute(ms - up[i]) / up[i])
print (np.sum (ers) / len (ers))

In [ ]:
datas, up, low = dataLoading ('vrf_small')
ers = list()
for i, data in enumerate (datas):
    solver = Solver (data)
    NEH (solver)
    ms = solver.calculateMakespan()
    ers.append (np.absolute(ms - up[i]) / up[i])
print (np.sum (ers) / len (ers))

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
color = sns.color_palette()
sns.set_style ('darkgrid')
rc = {'font.sans-serif': 'SimHei',
      'axes.unicode_minus': False}
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn
ers = np.array (ers)
sns.distplot (ers, fit = norm)
(mu, sigma) = norm.fit (ers)
print ('\n mu = {:.2f} and sigma = {:.2f}\n'.format (mu, sigma))
plt.legend (['Normal dist. ($\mu$ {:.2f} and $\sigma=$ {:.2f})'.format (mu, sigma)], loc = 'best')
plt.ylabel ('Frequency', fontproperties='SimHei')
plt.title ('Accuracy', fontproperties='SimHei')

fig = plt.figure()
res = stats.probplot (ers, plot = plt)
plt.show ()

In [ ]:
datas, up, low = dataLoading ('vrf_small')
ers = list()
for i, data in enumerate (datas):
    solver = Solver (data)
    NEH (solver, False, 'AD')
    ms = solver.calculateMakespan()
    ers.append (np.absolute(ms - up[i]) / up[i])
print (np.sum (ers) / len (ers))
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
color = sns.color_palette()
sns.set_style ('darkgrid')
rc = {'font.sans-serif': 'SimHei',
      'axes.unicode_minus': False}
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn
ers = np.array (ers)
sns.distplot (ers, fit = norm)
(mu, sigma) = norm.fit (ers)
print ('\n mu = {:.2f} and sigma = {:.2f}\n'.format (mu, sigma))
plt.legend (['Normal dist. ($\mu$ {:.2f} and $\sigma=$ {:.2f})'.format (mu, sigma)], loc = 'best')
plt.ylabel ('Frequency', fontproperties='SimHei')
plt.title ('Accuracy', fontproperties='SimHei')

fig = plt.figure()
res = stats.probplot (ers, plot = plt)
plt.show ()

In [ ]:
import random
def localSearch(solver, local_optimum = True, tie_breaking = False):
    current_makespan = 0
    need_improve = True
    while need_improve:
        current_jobs = solver.permutation.copy()
        random.shuffle (current_jobs)
        for job in current_jobs:
            solver.permutation.remove (job)
            current_makespan = current_makespan if current_makespan != 0 else solver.calculateMakespan()
            solver.insertIntoBestPosition (job, tie_breaking)
            if solver.makespan < current_makespan:
                need_improve = True
                current_makespan = solver.makespan
            else:
                need_improve = False
        if local_optimum == False:
            break;
        

In [ ]:
solver = Solver(datas[2])

In [ ]:
solver.permutation = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
insertionNeighborhood (solver)

In [ ]:
solver.permutation

In [ ]:
solver.makespan

In [ ]:
import random
import numpy as np

In [ ]:
def rouletteWheelSelection (fitness, n):
    total_fit = np.sum (fitness)
    permutation = []
    for i in range (n):
        rand = random.random() * total_fit
        for k in range (len (fitness)):
            rand -= fitness[k]
            if rand <= 0:
                permutation.append (k + 1)
                break
        else:
            permutation.append (len (fitness) - 1)
    return permutation

In [ ]:
f = [1, 2, 1, 2, 3, 4, 5, 7, 8, 9, 2, 3, 4, 5, 6, 7, 3 , 5, 5, 5, 6 ,6 ,7 ,8]
rouletteWheelSelection (f, len(f))

In [ ]:
def tournamentSelection (fitness, tournament_size, n):
    population = [x + 1 for x in range (len(fitness))]
    permutation = []
    for dummy in range (n):
        aspirants = random.sample (population, tournament_size)
        best = -1
        for asp in aspirants:
            if fitness[asp - 1] > best:
                best = fitness[asp - 1]
                winner = asp
        if best == 0:
            winner = random.choice (aspirants)
        permutation.append (winner)
        population.remove (winner)
    return permutation

In [ ]:
tournamentSelection(f, 2, int(len(f) / 2))

In [ ]:
def stochasticUniversalSampling(fitness, n):
    distance = np.sum (fitness) / n
    start = random.uniform(0, distance)
    points = [start + i * distance for i in range (n)]
    permutation = []
    for point in points:
        i = 0
        total_fitness = fitness[i]
        while total_fitness < point:
            i += 1
            total_fitness += fitness[i]
        permutation.append (i + 1)
    return permutation

In [ ]:
stochasticUniversalSampling (f, 15)

In [1]:
import math
import random
import numpy as np
from datetime import datetime, timedelta
from solver import Solver
import localSearch
import loadData
import NEH
import selection

In [18]:
class IteratedGreedy (object):
    def __init__ (self, data, temperature_init = 0.4, num_removed_jobs = 4, NEH_order = 'SD', tie_breaking = False, 
                  local_optimum = True, local_search = False, selection_algorithm = 'random', tournament_size = 5):
        self.current_solver = Solver (data)
        self.new_solver = Solver (data)
        self.best_solver = Solver (data)
        self.temperature_init = temperature_init
        self.num_removed_jobs = num_removed_jobs
        self.NEH_order = NEH_order
        self.tie_breaking = tie_breaking
        self.local_optimum = local_optimum
        self.local_search = local_search
        self.selection_algorithm = selection_algorithm
        self.tournament_size = tournament_size
    
    def eval (self, runtime):
        self.iterations = 0
        temperature = self.calculateTemperature()
        time_limit = datetime.now() + timedelta (milliseconds = runtime)
        NEH.NEH(self.current_solver, self.tie_breaking, self.NEH_order)
        localSearch.localSearch(self.current_solver, self.local_optimum, self.tie_breaking)
        self.best_solver.permutation = self.current_solver.permutation.copy()
        self.best_solver.makespan = self.current_solver.makespan
        while datetime.now() < time_limit:
            removed_jobs = self.selectJobsToRemove()
            self.new_solver.permutation = [x for x in self.current_solver.permutation if x not in removed_jobs]
            #print (removed_jobs)
            if self.local_search:
                localSearch.localSearch(self.new_solver, self.local_optimum, self.tie_breaking)
            for job in removed_jobs:
                self.new_solver.insertIntoBestPosition (job, self.tie_breaking)
            localSearch.localSearch(self.current_solver, self.local_optimum, self.tie_breaking)
            if self.new_solver.makespan < self.current_solver.makespan:
                self.current_solver.permutation = self.new_solver.permutation.copy()
                self.current_solver.makespan = self.new_solver.makespan
                if self.current_solver.makespan < self.best_solver.makespan:
                    self.best_solver.makespan = self.current_solver.makespan
                    self.best_solver.permutation = self.current_solver.permutation.copy()
            else:
                diff = self.new_solver.makespan - self.current_solver.makespan
                acceptance_probabilty = math.exp(- diff / temperature)
                if random.random() <= acceptance_probabilty:
                    self.current_solver.permutation = self.new_solver.permutation.copy()
                    self.current_solver.makespan = self.new_solver.makespan
            self.iterations += 1
        
    def computationalTime (self, runtime_facts):
        num = self.current_solver.n * (self.current_solver.m / 2)
        return num * runtime_facts
    
    def calculateTemperature (self):
        temperature = 0
        for i in range(self.current_solver.n):
            temperature += np.sum(self.current_solver.data[i])
        div = self.current_solver.n * self.current_solver.m * 10
        return self.temperature_init * (temperature / div)
    
    def selectJobsToRemove (self):
        if self.selection_algorithm == 'tournament':
            self.current_solver.calculateIdleTimes()
            fitness = self.current_solver.idle_times
            fitness[self.current_solver.permutation[0] - 1] = 0
            selected_jobs = selection.tournamentSelection (fitness, self.tournament_size, self.num_removed_jobs)
        elif self.selection_algorithm == 'roulette':
            self.current_solver.calculateIdleTimes()
            fitness = self.selectionFitness()
            fitness[self.current_solver.permutation[0] - 1] = 0
            selected_jobs = selection.rouletteWheelSelection (fitness, self.num_removed_jobs)
        elif self.selection_algorithm == 'stochastic_uni':
            self.current_solver.calculateIdleTimes()
            fitness = self.selectionFitness()
            fitness[self.current_solver.permutation[0] - 1] = 0
            selected_jobs = selection.stochasticUniversalSampling (fitness, self.num_removed_jobs)
        else:
            selected_jobs = random.sample(self.current_solver.permutation, self.num_removed_jobs)
        return selected_jobs
    
    def selectionFitness (self):
        fitness = np.empty (self.current_solver.n)
        for i in range(self.current_solver.n):
            total_fit = np.sum (self.current_solver.data[i])
            fitness[i] = (total_fit + self.current_solver.idle_times[i]) / total_fit
        return fitness

In [ ]:
def ex():
    instances, _, _ = loadData.dataLoading ("bit")
    for instance in instances:
        best_solver = IteratedGreedy(instance)
        best_solver.makespan = 100000000000000
        for i in range (10):
            ig = IteratedGreedy(instance)
            ig.selection_algorithm = "tournament"
            ig.local_search = True
            ig.tie_breaking = True
            ig.num_jobs_remove = 5
            ig.eval(25000)
            print (ig.best_solver.makespan)
            if ig.best_solver.makespan < best_solver.makespan:
                best_solver.permutation = ig.best_solver.permutation
                best_solver.makespan = ig.best_solver.makespan
        print("Best makespan", best_solver.makespan)
        print("Job permutation:", best_solver.permutation)

In [ ]:
ex()

In [ ]:
def exd():
    instances, _, _ = loadData.dataLoading ("bit")
    ig = IteratedGreedy(instances[-2])
    ig.local_search = True
    ig.tie_breaking = True
    ig.selection_algorithm = "tournament"
    ig.num_jobs_remove = 5
    ig.eval(25000)
    print("Best makespan", ig.best_solver.makespan,"iterations:", ig.iterations)
    print("Job permutation:", ig.best_solver.permutation)

In [ ]:
for i in range (10):
    exd()

In [ ]:
class Climb (object):
    def __init__ (self, data, NEH_order = 'SD', local_optimum = True, tie_breaking = False, max_loop = 10000):
        self.current_solver = Solver (data)
        self.new_solver = Solver (data)
        self.best_solver = Solver (data)
        self.tie_breaking = tie_breaking
        self.NEH_order = NEH_order
        self.local_optimum = local_optimum
        self.max_loop = max_loop
        
    def eval (self, runtime):
        self.iterations = 0
        time_limit = datetime.now() + timedelta (milliseconds = runtime)
        NEH.NEH(self.current_solver, self.tie_breaking, self.NEH_order)
        localSearch.localSearch(self.current_solver, self.local_optimum, self.tie_breaking)
        self.best_solver.permutation = self.current_solver.permutation.copy()
        self.best_solver.makespan = self.current_solver.makespan
        for iteration in range (self.max_loop):
            if datetime.now() >= time_limit:
                break;
            i = random.randint (0, len (self.current_solver.permutation) - 1)
            j = random.randint (0, len (self.current_solver.permutation) - 1)
            while i == j:
                j = random.randint (0, len (self.current_solver.permutation) - 1)
            self.new_solver.permutation = self.current_solver.permutation.copy()
            tmp = self.new_solver.permutation[i];
            self.new_solver.permutation[i] = self.new_solver.permutation[j];
            self.new_solver.permutation[j] = tmp;
            localSearch.localSearch(self.new_solver, self.local_optimum, self.tie_breaking)
            if self.new_solver.makespan < self.current_solver.makespan:
                self.current_solver.permutation = self.new_solver.permutation.copy()
                self.current_solver.makespan = self.new_solver.makespan
                if self.current_solver.makespan < self.best_solver.makespan:
                    self.best_solver.makespan = self.current_solver.makespan
                    self.best_solver.permutation = self.current_solver.permutation.copy()
            self.iterations += 1

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
color = sns.color_palette()
sns.set_style ('darkgrid')
rc = {'font.sans-serif': 'SimHei',
          'axes.unicode_minus': False}
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn
def ex():
    instances, up , _ = loadData.dataLoading ("et")
    accuracy = list()
    for index, instance in enumerate(instances):
        best_solver = Climb(instance)
        best_solver.makespan = 100000000000000
        for i in range (10):
            ig = Climb(instance)
            ig.tie_breaking = True
            ig.max_loop = 5000
            ig.eval(5000)
            # print (ig.best_solver.makespan)
            if ig.best_solver.makespan < best_solver.makespan:
                best_solver.permutation = ig.best_solver.permutation
                best_solver.makespan = ig.best_solver.makespan
                best_solver.iterations = ig.iterations
        accuracy.append (np.absolute (best_solver.makespan - up[index]) / up[index])
        print("Best makespan", best_solver.makespan, best_solver.iterations)
        print("Job permutation:", best_solver.permutation)
    auc = np.array (accuracy)
    sns.distplot (auc, fit = norm)
    (mu, sigma) = norm.fit (auc)
    print ('\n mu = {:.2f} and sigma = {:.2f}\n'.format (mu, sigma))
    plt.legend (['Normal dist. ($\mu$ {:.2f} and $\sigma=$ {:.2f})'.format (mu, sigma)], loc = 'best')
    plt.ylabel ('Frequency', fontproperties='SimHei')
    plt.title ('Accuracy', fontproperties='SimHei')

    fig = plt.figure()
    res = stats.probplot (auc, plot = plt)
    plt.show ()

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
color = sns.color_palette()
sns.set_style ('darkgrid')
rc = {'font.sans-serif': 'SimHei',
          'axes.unicode_minus': False}
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn
err_up = list()
err_low = list()
size_n = list()
size_m = list()
def ex():
    instances, up , low = loadData.dataLoading ("vrf_large")
    for index, instance in enumerate(instances):
        ig = SimulatedAnnealing(instance)
        ig.local_search = True
        ig.tie_breaking = True
        ig.max_loop = 100000
        ig.iteration_per_epoch = 1000
        ig.eval(40000)
        size_n.append (instance.shape[0])
        size_m.append (instance.shape[1])
        err_up.append ((ig.best_solver.makespan - up[index]) / up[index])
        err_low.append ((ig.best_solver.makespan - low[index]) / low[index])
        print (index, ig.iterations)
        #print("Best makespan", ig.best_solver.makespan, ig.iterations)
        #print("Job permutation:", ig.best_solver.permutation)
    ERR_up = np.array (err_up)
    sns.distplot (err_up, fit = norm)
    (mu, sigma) = norm.fit (ERR_up)
    print ('\n mu = {:.2f} and sigma = {:.2f}\n'.format (mu, sigma))
    plt.legend (['Normal dist. ($\mu$ {:.2f} and $\sigma=$ {:.2f})'.format (mu, sigma)], loc = 'best')
    plt.ylabel ('Frequency', fontproperties='SimHei')
    plt.title ('Error of UP', fontproperties='SimHei')

    fig = plt.figure()
    res = stats.probplot (ERR_up, plot = plt)
    plt.show ()
    ERR_low = np.array (err_low)
    sns.distplot (ERR_low, fit = norm)
    (mu, sigma) = norm.fit (ERR_low)
    print ('\n mu = {:.2f} and sigma = {:.2f}\n'.format (mu, sigma))
    plt.legend (['Normal dist. ($\mu$ {:.2f} and $\sigma=$ {:.2f})'.format (mu, sigma)], loc = 'best')
    plt.ylabel ('Frequency', fontproperties='SimHei')
    plt.title ('Error of LOW', fontproperties='SimHei')

    fig = plt.figure()
    res = stats.probplot (ERR_low, plot = plt)
    plt.show ()
    print (len(err_up), len(err_low), len(size_n), len(size_m))
    data = pd.DataFrame ({'err_up': err_up, 'err_low' : err_low, 'n' : size_n, 'm' : size_m})
    data.to_csv ('results/SA_vrf_large_ML1e5_IPE1000_RT40S.csv')

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
color = sns.color_palette()
sns.set_style ('darkgrid')
rc = {'font.sans-serif': 'SimHei',
          'axes.unicode_minus': False}
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn
err_up = list()
err_low = list()
size_n = list()
size_m = list()
def ex1():
    instances, up , low = loadData.dataLoading ("vrf_small")
    for index, instance in enumerate(instances):
        ig = SimulatedAnnealing(instance)
        ig.local_search = True
        ig.tie_breaking = True
        ig.max_loop = 100000
        ig.iteration_per_epoch = 1000
        ig.eval(40000)
        size_n.append (instance.shape[0])
        size_m.append (instance.shape[1])
        err_up.append ((ig.best_solver.makespan - up[index]) / up[index])
        err_low.append ((ig.best_solver.makespan - low[index]) / low[index])
        print (index, ig.iterations)
        #print("Best makespan", ig.best_solver.makespan, ig.iterations)
        #print("Job permutation:", ig.best_solver.permutation)
    ERR_up = np.array (err_up)
    sns.distplot (err_up, fit = norm)
    (mu, sigma) = norm.fit (ERR_up)
    print ('\n mu = {:.2f} and sigma = {:.2f}\n'.format (mu, sigma))
    plt.legend (['Normal dist. ($\mu$ {:.2f} and $\sigma=$ {:.2f})'.format (mu, sigma)], loc = 'best')
    plt.ylabel ('Frequency', fontproperties='SimHei')
    plt.title ('Error of UP', fontproperties='SimHei')

    fig = plt.figure()
    res = stats.probplot (ERR_up, plot = plt)
    plt.show ()
    ERR_low = np.array (err_low)
    sns.distplot (ERR_low, fit = norm)
    (mu, sigma) = norm.fit (ERR_low)
    print ('\n mu = {:.2f} and sigma = {:.2f}\n'.format (mu, sigma))
    plt.legend (['Normal dist. ($\mu$ {:.2f} and $\sigma=$ {:.2f})'.format (mu, sigma)], loc = 'best')
    plt.ylabel ('Frequency', fontproperties='SimHei')
    plt.title ('Error of LOW', fontproperties='SimHei')

    fig = plt.figure()
    res = stats.probplot (ERR_low, plot = plt)
    plt.show ()
    print (len(err_up), len(err_low), len(size_n), len(size_m))
    data = pd.DataFrame ({'err_up': err_up, 'err_low' : err_low, 'n' : size_n, 'm' : size_m})
    data.to_csv ('results/SA_vrf_small_ML1e5_IPE1000_RT40S.csv')

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
color = sns.color_palette()
sns.set_style ('darkgrid')
rc = {'font.sans-serif': 'SimHei',
          'axes.unicode_minus': False}
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn
err_up = list()
err_low = list()
size_n = list()
size_m = list()
def ex2():
    instances, up , low = loadData.dataLoading ("vrf_large")
    for index, instance in enumerate(instances):
        ig = Climb(instance)
        ig.tie_breaking = True
        ig.max_loop = 100000
        ig.eval(40000)
        size_n.append (instance.shape[0])
        size_m.append (instance.shape[1])
        err_up.append ((ig.best_solver.makespan - up[index]) / up[index])
        err_low.append ((ig.best_solver.makespan - low[index]) / low[index])
        print (index, ig.iterations)
        #print("Best makespan", ig.best_solver.makespan, ig.iterations)
        #print("Job permutation:", ig.best_solver.permutation)
    ERR_up = np.array (err_up)
    sns.distplot (err_up, fit = norm)
    (mu, sigma) = norm.fit (ERR_up)
    print ('\n mu = {:.2f} and sigma = {:.2f}\n'.format (mu, sigma))
    plt.legend (['Normal dist. ($\mu$ {:.2f} and $\sigma=$ {:.2f})'.format (mu, sigma)], loc = 'best')
    plt.ylabel ('Frequency', fontproperties='SimHei')
    plt.title ('Error of UP', fontproperties='SimHei')

    fig = plt.figure()
    res = stats.probplot (ERR_up, plot = plt)
    plt.show ()
    ERR_low = np.array (err_low)
    sns.distplot (ERR_low, fit = norm)
    (mu, sigma) = norm.fit (ERR_low)
    print ('\n mu = {:.2f} and sigma = {:.2f}\n'.format (mu, sigma))
    plt.legend (['Normal dist. ($\mu$ {:.2f} and $\sigma=$ {:.2f})'.format (mu, sigma)], loc = 'best')
    plt.ylabel ('Frequency', fontproperties='SimHei')
    plt.title ('Error of LOW', fontproperties='SimHei')

    fig = plt.figure()
    res = stats.probplot (ERR_low, plot = plt)
    plt.show ()
    print (len(err_up), len(err_low), len(size_n), len(size_m))
    data = pd.DataFrame ({'err_up': err_up, 'err_low' : err_low, 'n' : size_n, 'm' : size_m})
    data.to_csv ('results/Climb_vrf_large_ML1e5_RT40S.csv')

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
color = sns.color_palette()
sns.set_style ('darkgrid')
rc = {'font.sans-serif': 'SimHei',
          'axes.unicode_minus': False}
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn
err_up = list()
err_low = list()
size_n = list()
size_m = list()
def ex3():
    instances, up , low = loadData.dataLoading ("vrf_small")
    for index, instance in enumerate(instances):
        ig = Climb(instance)
        ig.tie_breaking = True
        ig.max_loop = 100000
        ig.eval(40000)
        size_n.append (instance.shape[0])
        size_m.append (instance.shape[1])
        err_up.append ((ig.best_solver.makespan - up[index]) / up[index])
        err_low.append ((ig.best_solver.makespan - low[index]) / low[index])
        print (index, ig.iterations)
        #print("Best makespan", ig.best_solver.makespan, ig.iterations)
        #print("Job permutation:", ig.best_solver.permutation)
    ERR_up = np.array (err_up)
    sns.distplot (err_up, fit = norm)
    (mu, sigma) = norm.fit (ERR_up)
    print ('\n mu = {:.2f} and sigma = {:.2f}\n'.format (mu, sigma))
    plt.legend (['Normal dist. ($\mu$ {:.2f} and $\sigma=$ {:.2f})'.format (mu, sigma)], loc = 'best')
    plt.ylabel ('Frequency', fontproperties='SimHei')
    plt.title ('Error of UP', fontproperties='SimHei')

    fig = plt.figure()
    res = stats.probplot (ERR_up, plot = plt)
    plt.show ()
    ERR_low = np.array (err_low)
    sns.distplot (ERR_low, fit = norm)
    (mu, sigma) = norm.fit (ERR_low)
    print ('\n mu = {:.2f} and sigma = {:.2f}\n'.format (mu, sigma))
    plt.legend (['Normal dist. ($\mu$ {:.2f} and $\sigma=$ {:.2f})'.format (mu, sigma)], loc = 'best')
    plt.ylabel ('Frequency', fontproperties='SimHei')
    plt.title ('Error of LOW', fontproperties='SimHei')

    fig = plt.figure()
    res = stats.probplot (ERR_low, plot = plt)
    plt.show ()
    print (len(err_up), len(err_low), len(size_n), len(size_m))
    data = pd.DataFrame ({'err_up': err_up, 'err_low' : err_low, 'n' : size_n, 'm' : size_m})
    data.to_csv ('results/Climb_vrf_small_ML1e5_RT40S.csv')

In [ ]:
ex()
ex1()
ex2()
ex3()

In [ ]:
instances, up , _ = loadData.dataLoading ("et")
size_n = [x.shape[0] for x in instances]
size_m = [x.shape[1] for x in instances]

In [ ]:
import pandas as pd
data = pd.DataFrame ({'auc': accuracy, 'n' : size_n, 'm' : size_m})


In [ ]:
data.to_csv ('results/Climb_et_ML5000_RT10S.csv')

In [ ]:
ERR_n = data.groupby('n')['auc'].mean()
s_n = data.groupby('n')['n'].mean()

In [ ]:
fig, ax = plt.subplots()
ax.scatter (y = ERR_n, x = s_n)
plt.xlabel ('n', fontproperties='SimHei')
plt.ylabel ('err_n', fontproperties='SimHei')
plt.show()

In [ ]:
class NawazEnscoreHam (object):
    def __init__ (self, data, order = 'SD', tie_breaking = False):
        self.solver = Solver (data)
        self.data = data
        self.order = order
        self.tie_breaking = tie_breaking
    def eval (self):
        NEH.NEH (self.solver, self.tie_breaking, self.order)
    

In [ ]:
auc

In [ ]:
def ex():
    instances, _, _ = loadData.dataLoading ("bit")
    for instance in instances:
        best_solver = Climb(instance)
        best_solver.makespan = 100000000000000
        for i in range (10):
            ig = NawazEnscoreHam(instance)
            ig.tie_breaking = True
            ig.eval()
            print (ig.solver.makespan)
            if ig.solver.makespan < best_solver.makespan:
                best_solver.permutation = ig.solver.permutation
                best_solver.makespan = ig.solver.makespan
        print("Best makespan", best_solver.makespan)
        print("Job permutation:", best_solver.permutation)

In [ ]:
ex()

In [ ]:
class SimulatedAnnealing (object):
    def __init__ (self, data, NEH_order = 'SD', local_optimum = True, tie_breaking = False, 
                  temperature_init = 0.4, annealing_rate = 0.99, iteration_per_epoch = 1000,
                  max_loop = 10000):
        self.current_solver = Solver (data)
        self.new_solver = Solver (data)
        self.best_solver = Solver (data)
        self.tie_breaking = tie_breaking
        self.NEH_order = NEH_order
        self.local_optimum = local_optimum
        self.temperature_init = temperature_init
        self.annealing_rate = annealing_rate
        self.iteration_per_epoch = iteration_per_epoch
        self.max_loop = max_loop
    
    def eval (self, runtime):
        self.iterations = 0
        time_limit = datetime.now() + timedelta (milliseconds = runtime)
        temperature = self.calculateInitialTemperature()
        NEH.NEH(self.current_solver, self.tie_breaking, self.NEH_order)
        localSearch.localSearch(self.current_solver, self.local_optimum, self.tie_breaking)
        self.best_solver.permutation = self.current_solver.permutation.copy()
        self.best_solver.makespan = self.current_solver.makespan
        for iteration in range (self.max_loop):
            if datetime.now() >= time_limit:
                break
            i = random.randint (0, len (self.current_solver.permutation) - 1)
            j = random.randint (0, len (self.current_solver.permutation) - 1)
            while i == j:
                j = random.randint (0, len (self.current_solver.permutation) - 1)
            self.new_solver.permutation = self.current_solver.permutation.copy()
            tmp = self.new_solver.permutation[i];
            self.new_solver.permutation[i] = self.new_solver.permutation[j];
            self.new_solver.permutation[j] = tmp;
            localSearch.localSearch(self.new_solver, self.local_optimum, self.tie_breaking)
            if self.new_solver.makespan < self.current_solver.makespan:
                self.current_solver.permutation = self.new_solver.permutation.copy()
                self.current_solver.makespan = self.new_solver.makespan
                if self.current_solver.makespan < self.best_solver.makespan:
                    self.best_solver.makespan = self.current_solver.makespan
                    self.best_solver.permutation = self.current_solver.permutation.copy()
            else:
                diff = self.new_solver.makespan - self.current_solver.makespan
                acceptance_probabilty = math.exp(- diff / temperature)
                if random.random() <= acceptance_probabilty:
                    self.current_solver.permutation = self.new_solver.permutation.copy()
                    self.current_solver.makespan = self.new_solver.makespan
            self.iterations += 1
            if self.iterations % self.iteration_per_epoch == 0:
                temperature *= self.annealing_rate
        
    def calculateInitialTemperature (self):
        temperature = 0
        for i in range(self.current_solver.n):
            temperature += np.sum(self.current_solver.data[i])
        div = self.current_solver.n * self.current_solver.m * 10
        return self.temperature_init * (temperature / div)
            

In [ ]:
def exd():
    instances, _, _ = loadData.dataLoading ("bit")
    for instance in instances:
        ig = SimulatedAnnealing(instance)
        ig.local_search = True
        ig.tie_breaking = True
        ig.max_loop = 10000
        ig.iteration_per_epoch = 500
        ig.eval(25000)
        print("Best makespan", ig.best_solver.makespan,"iterations:", ig.iterations)
        print("Job permutation:", ig.best_solver.permutation)

In [ ]:
exd()